# Importing the code, numpy and graphics utilities

In [1]:
from sys import path
path.append('./src/')
path.append('./moduls/')

import fimera as chimera
from species import Specie
from solvers import Solver
from chimera_main import ChimeraRun
from diagnostics import Diagnostics

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib
mymap1 = plt.cm.seismic;mymap1._init()
mymap1._lut[:,-1] = abs(np.sin(np.r_[-0.5*np.pi:0.5*np.pi:259j]))
mymap2 = plt.cm.PiYG;mymap2._init()
mymap2._lut[:,-1] = abs(np.sin(np.r_[-0.5*np.pi:0.5*np.pi:259j]))**2

Using matplotlib backend: MacOSX


# FEL SIMULATION

# Defining the simulation parameters

## Undulator

In [ ]:
K0 = 1.95
Periods = 350
lam0 = 2.8

## Electron beam

In [ ]:
g0 = 200/0.511
lbx  = 80e-4/lam0
lbr  = 90e-4/lam0

## Simulation grid

In [ ]:
Lgx  = 200e-4  /lam0
Rg   = 2000e-4  /lam0
Rg_cut = 800e-4/lam0

## Preparing input

In [ ]:
dt = 1./40
Steps2Do = int(Periods/dt)+1

BoxGrid     = (-0.5*Lgx, 0.5*Lgx, Rg, Lgx/80., Rg/80.)
BoxGridBeam = (-0.5*Lgx, 0.5*Lgx, Rg, Lgx/80., lbr/50.)
Domain = (-0.5*lbx, 0.5*lbx, 0.0, lbr)

k_res = 2*g0**2/(1+K0**2/2)
gg = g0/(1.+K0**2/2)**.5
vb = (1.-gg**-2)**0.5

micro_scl = lam0*1e4
nano_scl = lam0*1e7

## Arguments for the simulation components

In [ ]:
beam_in = {
    'Charge':-1.,'Mass':1.,'Density':56.6, 'MomentaMeans':(g0,0.,0.), 'MomentaSpreads':(1e-4*g0,0.008,0.008),
    'Grid':BoxGridBeam, 'TimeStep':dt,'RandCell':30,'Xchunked':(8,6),
    'Devices':([chimera.undul_analytic,np.array([K0, 1., 1., 1000.])],)
}

solver_in = {
    'Grid':BoxGrid, 'TimeStep':dt, 'MaxAzimuthMode':2, 'KxShift':k_res,'CoPropagative':vb,
    'Rcut':800e-4/lam0 ,'Xchunked':(8,6)
}

seed_in = {
    'a0':0.107,'k0':k_res,'x0':-30e-4/lam0,'x_foc':70.0/lam0,'Lx':15e-4/lam0,'LR':180e-4/lam0
}

MovingFrame = {
    'TimeStep':dt,'Steps':1,'Velocity':vb,'Features':('Staged','NoSorting')
}

## Constructing the simulation components

In [ ]:
specie = Specie(beam_in)
specie.add_particles(specie.gen_parts(Domain=Domain))
specie.denoise((k_res,))
solver = Solver(solver_in)
solver.add_gauss_beam(seed_in)

chimera_in = {'Solvers':(solver,),'Particles':(specie,),'MovingFrames':(MovingFrame,)}
Chimera = ChimeraRun(chimera_in)

Diags = Diagnostics(Chimera,(),out_folder=None)

# Running the simulation

In [ ]:
Chimera.make_halfstep()
for i in xrange(Steps2Do): Chimera.make_step(i)

# Animated run example

In [ ]:
Chimera.make_halfstep()
nrg0 = Diags.nrg_out({'Features':('Return',)})[0]*lam0*1e6
pwr0 = Diags.pwr_out({'Features':('Return','Spot')})[0]
xx = solver.Args['Xgrid']*micro_scl
rr = solver.Args['RgridFull'][None,1:]*micro_scl*1e-3
oo = np.r_[0:2*np.pi:60j][:,None]

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(16,16),dpi=90)

diag_nrg = [nrg0.sum(),]
pl1, = ax1.semilogy([0,],diag_nrg,)
ax1.set_xlim(0,1.1*Periods*micro_scl*1e-6)
ax1.set_ylim(1e-4,1e2)
ax1.set_title('Radiation energy',size=16)
ax1.set_xlabel('distance (m)',size=16)
ax1.set_ylabel('Energy ($\mu$J)',size=16)

pl2 = ax2.pcolormesh(rr*np.sin(oo),rr*np.cos(oo),
                     Diags.pwr_out({'Features':('Return','Spot')})[0][1],
                     cmap='hot',shading='gouraud')
ax2.set_xlim(-0.38,0.38)
ax2.set_ylim(-0.38,0.38)
ax2.set_title('Intensity profile',size=16)
ax2.set_xlabel('x (mm)',size=16)
ax2.set_ylabel('y (mm)',size=16)

pl3, = ax3.plot(xx,pwr0[0]*1e-6,)
ax3.set_title('Power profile',size=16)
ax3.set_xlabel('z ($\mu$m)',size=16)
ax3.set_ylabel('P (MW)',size=16)

pl4 = ax4.imshow(chimera.density_2x(specie.particles[1]*lam0*10,
                 specie.particles[2]*lam0*10,specie.particles[-1],
                 (-0.4,0.4,-0.4,0.4),200,200), origin='lower',
                 aspect='auto',cmap='hot_r',
                 extent=(-0.4,0.4,-0.4,0.4))
ax4.set_xlim(-0.38,0.38)
ax4.set_ylim(-0.38,0.38)
ax4.set_title('Beam profile',size=16)
ax4.set_xlabel('x (mm)',size=16)
ax4.set_ylabel('y (mm)',size=16)

StepAnim = int(1./dt/6)

def animate(i):
    for il in xrange(i*StepAnim,(i+1)*StepAnim):
        Chimera.make_step(il)
    pwr0 = Diags.pwr_out({'Features':('Return','Spot')})[0]
    nrg0 = Diags.nrg_out({'Features':('Return',)})[0]*lam0*1e6
    dens = chimera.density_2x(specie.particles[1]*lam0*10,
        specie.particles[2]*lam0*10,specie.particles[-1],
                 (-0.4,0.4,-0.4,0.4),200,200)

    diag_nrg.append(nrg0.sum())
    pl1.set_xdata(np.arange(len(diag_nrg))*StepAnim*dt*micro_scl*1e-6)
    pl1.set_ydata(diag_nrg)

    pl2.set_array(pwr0[1].ravel())
    pl2.set_clim(0,pwr0[1].max())

    pl3.set_ydata(pwr0[0]*1e-6)
    ax3.set_ylim(0,1.1*pwr0[0].max()*1e-6)

    pl4.set_data(dens)
    pl4.set_clim(dens.min(),0)

    return pl1,

In [ ]:
%timeit animate(0)

In [ ]:
ani = animation.FuncAnimation(fig, animate,frames=int(Periods/dt/StepAnim),blit=True,repeat=False)
#plt.show()

In [ ]:
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'
FFwriter = animation.FFMpegWriter(fps=20)
ani.save('basic_animation.mp4', writer = FFwriter)

# LASER PLASMA ACCELERATION PIC SIMULATION

## Preparing the input

In [13]:
dt = 0.04
Steps2Do = 4001
BoxGrid = (-50.,0.0,16.,0.04,0.3)
densprof = lambda x: np.interp(x, [0.,40.,50.,300],[0.,1.,0.5,0.5])

## Arguments for the simulation components

In [14]:
electrons_in = {
    'Charge':-1., 'Mass':1., 'Density':0.005, 
    'Grid':BoxGrid, 'TimeStep':dt,'FixedCell':(2,2,3), 'Xchunked':(4,41),'Features':('NoSorting',)
}

ions_in = {
    'Charge':1.,'Mass':1886., 'Density':0.005, 
    'Grid':BoxGrid, 'TimeStep':dt,'FixedCell':(3,3,4),'Xchunked':(4,41), 'Features':('NoSorting','Still')
}

solver_in = {
    'Grid':BoxGrid,'TimeStep':dt, 'MaxAzimuthMode':1,
    'Xchunked':(4,41),'Features':('SpaceCharge','StillAsBackground')
}

laser_in = {
    'a0':3.0,'k0':1.0,'x0':-14.0,'x_foc':45.0,'Lx':4,'LR':4
}

MovingFrame = {
    'TimeStep':dt,'Steps':40,'AbsorbLayer':10.,'AddPlasma':densprof
}

## Constructing the simulation components

In [15]:
solver = Solver(solver_in)
solver.add_gauss_beam(laser_in)
electrons = Specie(electrons_in)
ions = Specie(ions_in)

chimera_in = {
    'Solvers':(solver,),'Particles':(electrons,ions),'MovingFrames':(MovingFrame,)
}

Chimera = ChimeraRun(chimera_in)
Diags = Diagnostics(Chimera,(),out_folder=None)

Constructing a solver in the cylindric box (-50.0, 0.0, 16.0)
spatial and temporal resolutions are (0.04, 0.3, 0.04)
Grid resolutions are  (1248, 53, 2)
Space charge is added
"Still" species as are treated as background


# Animated run example

In [16]:
Chimera.make_halfstep()

fig, ax1 = plt.subplots(figsize=(12,12),dpi=120)
StepAnim = 40

ee = Diags.fld_out({'Features':{'Return',},})[0]
dens = Diags.dns_out({'Features':{'Return':True,'MaxMode':1}})[0]
extnt = np.array([solver.Args['leftX'],solver.Args['rightX'],
                  -solver.Args['Rgrid'][-1],solver.Args['Rgrid'][-1]])
ax1.set_xlim(extnt[0]+10.,extnt[1])

laser = np.real(np.hstack((ee[:,::-1,0,2],ee[:,1:,0,2])))+\
2*np.real(np.hstack((ee[:,::-1,1,2],-ee[:,1:,1,2])))
eons = np.real(np.hstack((dens[:,::-1,0],dens[:,1:,0])))+\
2*np.real(np.hstack((dens[:,::-1,1],-dens[:,1:,1])))

pl1 = ax1.imshow(np.abs(eons).T, origin='lower',aspect='auto',
        extent=extnt,cmap='bone',interpolation='lanczos',vmax=0.01)
pl3 = ax1.imshow(laser.T, origin='lower',aspect='auto',
                 extent=extnt,cmap=mymap2)

def animate(i):
    for il in xrange(i*StepAnim,(i+1)*StepAnim):
        Chimera.make_step(il)
    ee = Diags.fld_out({'Features':{'Return',},})[0]
    dens = Diags.dns_out({'Features':{'Return':True,'MaxMode':1}})[0]
    extnt = np.array([solver.Args['leftX'],solver.Args['rightX'],
                  -solver.Args['Rgrid'][-1],solver.Args['Rgrid'][-1]])

    laser = np.real(np.hstack((ee[:,::-1,0,2],ee[:,1:,0,2])))+\
      2*np.real(np.hstack((ee[:,::-1,1,2],-ee[:,1:,1,2])))
    eons = np.real(np.hstack((dens[:,::-1,0],dens[:,1:,0])))+\
      2*np.real(np.hstack((dens[:,::-1,1],-dens[:,1:,1])))
    extnt[:2] += StepAnim*dt
    pl1.set_data(np.abs(eons).T);pl1.set_extent(extnt);pl1.set_clim(0,0.01)
    pl3.set_data(laser.T);pl3.set_extent(extnt);pl3.set_clim(-3,3)
    ax1.set_xlim(extnt[0]+10.,extnt[1])
    return pl1,

adding inits


In [ ]:
%timeit animate(0)

In [17]:
ani = animation.FuncAnimation(fig, animate,frames=int(Steps2Do/StepAnim),repeat=False)
#plt.show()

In [18]:
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'
FFwriter = animation.FFMpegWriter(fps=10)
ani.save('basic_animation1.mp4', writer = FFwriter)